In [ ]:
#@title Dialogflow Authentication { display-mode: 'form' }

def setup():
    !gcloud auth login
    !gcloud config set project demobot1-madh

setup()

In [ ]:
#@title Data description { display-mode: 'form' }

cols = [
    'SHAPE',
    'COLOR',
    'FLUOR',
    'PURITY',       # Clarity
    "WEIGHT",
    'CUT',
    'POLISH',
    'SYMN',
    #
    'MES1',
    'MES2',
    'MES3',
    'TABLE',
    'DEPTHPER',
    'PRICE/CTS',
    'TOTAL',
    'RATIO',
    #
    "REPORTNO",
    "VIDEOLINK",
    "PDFLINK",
    "CSTATUS",
    "CERT"
]
def process_cols(c):
    if len(c) == 0:
        return cols
    return c

shape = {
    'RD': ['R', 'BR', 'BRILLIANT CUT', 'BRILLIANTCUT', 'RB', 'ROUND', 'RD', 'ROUND BRILLIANT', 'BRILLIANT', 'ROUNDBRILLIANT'],
    'OV': ['OVEL', 'OVAL', 'OV', 'OC', 'OL'],
    'EM': ['EMERALD', 'EMRD', 'EM', 'EC', 'EMD'],
    'CU': ['CS', 'CUSHIONMODIFIED', 'CU', 'CMB', 'CM', 'CUSHION', 'CUS', 'CUSHION MODIFIED'],
    'PR': ['PRINCESS', 'PC', 'PR', 'PRINCE'],
    'PS': ['PAER', 'PER', 'PEAR', 'PS'],
    'RA': ['RAD', 'RADIANT', 'RA'],
    'MQ': ['MR', 'MAR', 'MARQUISE', 'MQ'],
    'AS': ['AS', 'ASHCHER', 'ASSCHER'],
    'HS': ['HS', 'HC', 'HEART', 'HR', 'LOVE', 'HRT'],
    'TR': ['TRI', 'TR', 'TRIANGLE']
}
def process_shape(s):
    res = []

    for sh in s:
        for sha in shape:
            if sh in shape[sha]:
                res.append(sha)
                break

    return res

def process_weight(wh, diff = 0.15):
    res = []

    if len(wh) == 1:
        w = wh[0]
        w = w.replace('ct', '').replace('carat', '').replace('CT', '').replace('Ct', '').replace('CARAT', '').replace('Carat', '')
        w = w.replace('to', '-').replace(' ', '').split('-')

        if len(w) == 2:
            res.append([float(w[0]), float(w[1])])
        else:
            i = float(w[0])
            res.append([i - diff, i + diff])
    else:
        for w in wh:
            res.append(process_weight([w]))
    
    return res

def process_color(cl):
    res = []

    if len(cl) == 1:
        c = cl[0]
        c = c.replace('between', '').replace('Between', '').replace('BETWEEN', '')
        c = c.replace('color', '').replace('Color', '').replace('COLOR', '').replace(' ', '')
        c = c.replace('to', '-').replace('and', '-').split('-')

        if len(c) == 2:
            for a in range(ord(c[0]), ord(c[1]) + 1):
                res.append(chr(a))
        else:
            res.append(list(c[0]))
    else:
        try:
            for a in range(ord(cl[0]), ord(cl[1]) + 1):
                try:
                    res.append(chr(a))
                except:
                    pass
        except:
            for a in cl:
                res.append(process_color([a]))

    return res

def process_cps(cps):
    res = []

    if isinstance(cps, list):
        for i in cps:
            res.append(process_cps(i))
    else:
        i = cps.replace('cut', '').replace('Cut', '').replace('CUT', '')
        i = i.replace('symn', '').replace('Symn', '').replace('SYMN', '')
        i = i.replace('polish', '').replace('Polish', '').replace('POLISH', '')
        i = i.replace('symmetry', '').replace('Symmetry', '').replace('SYMMETRY', '')
    
        res.append(i)
    return res

# API Integration

In [ ]:
import requests
import subprocess

auth = subprocess.check_output('gcloud auth print-access-token', shell = True)[:-1].decode("utf-8")

def post(text, lang, session = '1689', project = 'demobot1-madh'):
    url = f'https://dialogflow.googleapis.com/v2/projects/{project}/agent/sessions/{session}:detectIntent'
    headers = {
        'Authorization': f'Bearer {auth}',
        'x-goog-user-project': project,
        'Content-Type': 'application/json; charset=utf-8'
    }
    payload = {
        "query_input": {
            "text": {
                "text": text,
                "language_code": lang
            }
        }
    }

    res = requests.post(url, json = payload, headers = headers)

    return res.text

In [ ]:
#@title Get user query { display-mode: 'form' }
import json
import numpy as np

fallback = [
    'I didn\'t get that. Can you say it again?',
    'I missed what you said. What was that?',
    'Sorry, could you say that again?',
    'Sorry, can you say that again?',
    'Can you say that again?',
    'Sorry, I didn\'t get that. Can you rephrase?',
    'Sorry, what was that?',
    'I didn\'t get that. Can you repeat?'
]

def query(text = None, lang = 'en-US', debug = False):
    try:
        if text is None:
            text = input("Enter a query to be processed:\n")
        
        response = json.loads(post(text, lang))
        if debug:
            print(json.dumps(response, indent = 4))

        extracted = response['queryResult']['parameters']

        try:
            cols = extracted['cols']
        except:
            cols = []
        # cols = process_cols(cols)

        try:
            shape = extracted['shape']
        except:
            shape = []
        shape = process_shape(shape)

        try:
            color = extracted['color']
        except:
            color = extracted['colorC']
        color = process_color(color)

        try:
            fluor = extracted['fluor']
        except:
            fluor = []
        
        try:
            clarity = extracted['clarity']
        except:
            clarity = []

        try:
            weight = extracted['weightC']
        except:
            try:
                weight = extracted['weight']
            except:
                try:
                    weight = extracted['diamond-weight']
                except:
                    weight = []
        weight = process_weight(weight)

        try:
            cut = extracted['cut']
        except:
            cut = []
        cut = process_cps(cut)

        try:
            polish = extracted['polish']
        except:
            polish = []
        polish = process_cps(polish)
        
        try:
            symmetry = extracted['symmetry']
        except:
            symmetry = []
        symmetry = process_cps(symmetry)

        print('-'*60)
        print('Dialogflow generated the following filter:')
        print()

        if len(cols) == 0:
            cols.append('REPORTNO')
            if len(shape):
                cols.append('SHAPE')
            if len(color):
                cols.append('COLOR')
            if len(fluor):
                cols.append('FLUOR')
            if len(clarity):
                cols.append('CLARITY')
            if len(weight):
                cols.append('WEIGHT')
            if len(cut):
                cols.append('CUT')
            if len(polish):
                cols.append('POLISH')
            if len(symmetry):
                cols.append('SYMN')
        
        if len(cols):
            print(f'\tColumns = {cols}')
        if len(shape):
            print(f'\tShape = {shape}')
        if len(color):
            print(f'\tColor = {color}')
        if len(fluor):
            print(f'\tFluoroscence = {fluor}')
        if len(clarity):
            print(f'\tClarity = {clarity}')
        if len(weight):
            print(f'\tWeight = {weight}')
        if len(cut):
            print(f'\tCut = {cut}')
        if len(polish):
            print(f'\tPolish = {polish}')
        if len(symmetry):
            print(f'\tSymmetry = {symmetry}')
    except:
        print('-'*60)

        print(np.random.choice(fallback))

# Query Processing

In [ ]:
# query(debug = True)

In [ ]:
query()

In [ ]:
query()